## 네이버 리뷰 데이터 크롤링

In [ ]:
#!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#리뷰 데이터 불러오기
total_rvs = pd.read_csv('crawling/total.csv', encoding='utf-8')
total_rvs = total_rvs.iloc[:,2:].drop_duplicates()
total_rvs.head()

,title,가고싶다,전체평점,주소,음식종류,locate
0,만가타,"2,799",4.8,서울시 종로구 소격동 88-17,세계음식 기타,북촌
36,갈로팡,385,4.6,서울시 종로구 계동 99-12 2F,프랑스 음식,북촌
47,일월카츠,121,4.6,서울시 종로구 계동 144-1,까스 요리,북촌
52,스시키즈나,868,4.5,서울시 종로구 소격동 159,회 / 스시,북촌
64,오레노라멘,"2,023",4.5,서울시 종로구 안국동 159 1F,라멘 / 소바 / 우동,북촌


In [ ]:
#필요 정보만 추출
stores = list(total_rvs['title'])
locates = list(total_rvs['locate'])

In [ ]:
def page_back():
    driver.back()
    time.sleep(2)
    try:
        driver.find_element_by_css_selector('div.PopupUseApp__linkSub').click()
        time.sleep(2)
    except:
        pass
    try:
        driver.find_element_by_css_selector('div.BottomBarUseApp__Close').click()
        time.sleep(2)
    except:
        pass

In [ ]:
total_2 = pd.DataFrame()
nan_list = []

In [ ]:
# 크롬으로 네이버 지도 열기
options = Options()
options.add_argument('--kiosk')
driver = webdriver.Chrome("./chromedriver.exe")
driver.get("https://m.map.naver.com/")
time.sleep(5)



for i in range(2):
    
    # 검색
    search_title = stores[i]
    search_locate = locates[i]
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input.Nbox_input_text._keyword"))
        )
    finally:
        pass
    input = driver.find_element_by_css_selector("input.Nbox_input_text._keyword")
    input.click()
    time.sleep(2)
    input = driver.find_element_by_css_selector("input.Nbox_input_text._search_input")
    input.send_keys(search_locate+' '+search_title)
    time.sleep(2)
    input.send_keys(Keys.ENTER)
    time.sleep(2)
    
    
    # 페이지 들어가기
    try:
        element = WebDriverWait(driver, 10).until(
           EC.presence_of_element_located((By.CSS_SELECTOR, 'a.a_item.a_item_distance._linkSiteview'))
       )
    except:
        total_2 = total_2.append(pd.Series(name=0))
        total_2.iloc[len(total_2)-1,0] = stores[i]
        nan_list.append(stores[i])
        driver.get("https://m.map.naver.com/")
        continue
    finally:
        pass
    driver.find_elements_by_css_selector("a.a_item.a_item_distance._linkSiteview")[0].click()
    
    
    # 리뷰 버튼 클릭
    try:
        element = WebDriverWait(driver, 10).until(
           EC.presence_of_element_located((By.CSS_SELECTOR, 'span._3aXen'))
       )
    except:
        total_2 = total_2.append(pd.Series(name=0))
        total_2.iloc[len(total_2)-1,0] = stores[i]
        nan_list.append(stores[i])
        driver.get("https://m.map.naver.com/")
        continue
    finally:
        pass
    button_list = driver.find_elements_by_css_selector('span._3aXen')
    for button in button_list:
        if button.text == '리뷰':
            button.click()
    driver.refresh()
    
    
    # 리뷰 더보기
    try:
        element = WebDriverWait(driver, 10).until(
           EC.presence_of_element_located((By.CSS_SELECTOR, 'a._22igH'))
       )
    except:
        total_2 = total_2.append(pd.Series(name=0))
        total_2.iloc[len(total_2)-1,0] = stores[i]
        nan_list.append(stores[i])
        driver.get("https://m.map.naver.com/")
        continue
    finally:
        pass
    more_btn = driver.find_element_by_css_selector('a._22igH')
    action = ActionChains(driver)
    while True:
        try:
            action.move_to_element(more_btn).perform()
            more_btn.click()
            time.sleep(2)
        except:
            break
    
    
    #딕셔너리 만들기
    temp = dict()
    temp['title'] = [search_title]

    try:
        element = WebDriverWait(driver, 10).until(
           EC.presence_of_element_located((By.CSS_SELECTOR, 'div._3ZEZK'))
       )
    except:
        total_2 = total_2.append(pd.Series(name=0))
        total_2.iloc[len(total_2)-1,0] = stores[i]
        nan_list.append(stores[i])
        driver.get("https://m.map.naver.com/")
        continue
    finally:
        pass
    if len(driver.find_elements_by_css_selector('div._8_WED')) > 0:
        total_2 = total_2.append(pd.Series(name=0))
        total_2.iloc[len(total_2)-1,0] = stores[i]
        nan_list.append(stores[i])
        driver.get("https://m.map.naver.com/")
        continue
    keywords = driver.find_elements_by_css_selector('div._3ZEZK')
    for w in keywords:
        kwd = w.find_element_by_css_selector('span._1lntw').text[1:-1]
        num = int(w.find_element_by_css_selector('span.Nqp-s').text[14:])
        temp[kwd] = [num]
    
    
    #데이터프레임에 추가
    total_2 = total_2.append(pd.DataFrame(temp))
    
    driver.get("https://m.map.naver.com/")

IndexError: iloc cannot enlarge its target object

In [ ]:
total2 = pd.read_csv('crawling/total_2.csv', encoding='utf-8')
total3 = pd.read_csv('crawling/total_3.csv', encoding='utf-8')

In [ ]:
total1 = pd.concat([total2, total3],ignore_index=True)
total1 = total1.drop(['Unnamed: 0'], axis=1)
total1

,title,음식이 맛있어요,인테리어가 멋져요,특별한 날 가기 좋아요,특별한 메뉴가 있어요,친절해요,매장이 청결해요,재료가 신선해요,가성비가 좋아요,뷰가 좋아요,...,조식이 맛있어요,즐길 거리가 많아요,대중교통이 편해요,냉난방이 잘돼요,방음이 잘돼요,수영장이 잘 되어있어요,신선해요,부위가 다양해요,포장이 깔끔해요,원산지 표시가 명확해요
0,만가타,169.0,147.0,138.0,129.0,116.0,104.0,71.0,41.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,갈로팡,31.0,1.0,24.0,18.0,28.0,18.0,16.0,11.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,일월카츠,80.0,19.0,5.0,13.0,72.0,50.0,38.0,11.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,스시키즈나,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,오레노라멘,565.0,25.0,8.0,151.0,212.0,117.0,76.0,89.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233,브레드덕,NaN,1.0,NaN,50.0,61.0,14.0,NaN,20.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2234,리틀크레이지피자,158.0,5.0,2.0,52.0,36.0,36.0,27.0,110.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2235,바른식시골보쌈감자옹심이,524.0,10.0,16.0,59.0,106.0,75.0,90.0,80.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2236,프라이밋 스테이크하우스,29.0,3.0,6.0,6.0,13.0,2.0,6.0,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
total1.to_csv('crawling/total_naver.csv', encoding='utf-8')

In [ ]:
total_2 = pd.read_csv('crawling/total_naver.csv', encoding='utf-8')
total_2 = total_2.drop('Unnamed: 0', axis=1)
total_2.head()

,title,음식이 맛있어요,인테리어가 멋져요,특별한 날 가기 좋아요,특별한 메뉴가 있어요,친절해요,매장이 청결해요,재료가 신선해요,가성비가 좋아요,뷰가 좋아요,...,조식이 맛있어요,즐길 거리가 많아요,대중교통이 편해요,냉난방이 잘돼요,방음이 잘돼요,수영장이 잘 되어있어요,신선해요,부위가 다양해요,포장이 깔끔해요,원산지 표시가 명확해요
0,만가타,169.0,147.0,138.0,129.0,116.0,104.0,71.0,41.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,갈로팡,31.0,1.0,24.0,18.0,28.0,18.0,16.0,11.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,일월카츠,80.0,19.0,5.0,13.0,72.0,50.0,38.0,11.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,스시키즈나,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,오레노라멘,565.0,25.0,8.0,151.0,212.0,117.0,76.0,89.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nan_list = []
for i in range(len(total_2)):
    if total_2.iloc[i].isnull().sum() == 75:
        nan_list.append([i, total_2.iloc[i]['title']])

## 네이버 리뷰 데이터 기반 점수 계산

In [ ]:
data = total_2.copy()
data = data.fillna(0)

In [ ]:
comments = data.columns[2:]
comments

In [ ]:
taste = ['음식이 맛있어요', '특별한 메뉴가 있어요', '재료가 신선해요', '단체모임 하기 좋아요', '커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '빵이 맛있어요', '차가 맛있어요', 
         '반찬이 잘 나와요', '비싼 만큼 가치있어요', '과채가 신선해요', '현지 맛에 가까워요', '직접 잘 구워줘요', '신선해요', '고기 질이 좋아요', '조식이 맛있어요']
price = ['가성비가 좋아요', '가격이 합리적이에요']
service = ['친절해요', '매장이 청결해요', '재료가 신선해요', '화장실이 깨끗해요', '양이 많아요', '매장이 넓어요', '혼밥하기 좋아요', '단체모임 하기 좋아요', '주차하기 편해요',
           '좌석이 편해요', '집중하기 좋아요', '반찬이 잘 나와요', '술이 다양해요', '기본 안주가 좋아요', '상품이 다양해요', '행사 상품이 다양해요', '과채가 신선해요', 
           '세트 구성이 알차요', '메뉴 구성이 알차요', '직접 잘 구워줘요', '음식이 빨리 나와요', '고기 질이 좋아요', '공간이 넓어요', '설명이 잘 되어있어요', '유익해요',
           '시설이 깔끔해요', '책 종류가 다양해요', '코스요리가 알차요', '깨끗해요', '호캉스하기 좋아요', '조용히 쉬기 좋아요', '즐길 거리가 많아요',
           '대중교통이 편해요', '냉난방이 잘돼요', '방음이 잘돼요', '수영장이 잘 되어있어요', '신선해요', '부위가 다양해요', '포장이 깔끔해요', '원산지 표시가 명확해요', 
           '관리가 잘 되어있어요', '규모가 커요', '공연이 재밌어요']
mood = ['인테리어가 멋져요', '특별한 날 가기 좋아요', '뷰가 좋아요', '사진이 잘 나와요', '대화하기 좋아요', '차분한 분위기에요', '오래 머무르기 좋아요', '음악이 좋아요', 
        '혼술하기 좋아요','선물하기 좋아요', '비주얼이 좋아요', '아늑해요', '책 읽기 쾌적해요', '카페 같은 분위기에요', '깨끗해요', '호캉스하기 좋아요', '조용히 쉬기 좋아요', 
        '야외 공간이 멋져요', '볼거리가 많아요', '관리가 잘 되어있어요', '컨셉이 독특해요']


In [ ]:
for i in range(len(taste)):
  if i == 0:
    taste_sum = data[taste[i]]
  else:
    taste_sum = taste_sum + data[taste[i]]

In [ ]:
for i in range(len(price)):
  if i == 0:
    price_sum = data[price[i]]
  else:
    price_sum = price_sum + data[price[i]]

In [ ]:
for i in range(len(service)):
  if i == 0:
    service_sum = data[service[i]]
  else:
    service_sum = service_sum + data[service[i]]

In [ ]:
for i in range(len(mood)):
  if i == 0:
    mood_sum = data[mood[i]]
  else:
    mood_sum = mood_sum + data[mood[i]]

In [ ]:
data['taste'] = taste_sum
data['price'] = price_sum
data['service'] = service_sum
data['mood'] = mood_sum

In [ ]:
for i in range(len(comments)):
  if i == 0:
    comments_sum = data[comments[i]]
  else:
    comments_sum = comments_sum + data[comments[i]]
data['total'] = comments_sum

In [ ]:
data['맛'] = round(data['taste']/data['total'], 2)
data['가성비'] = round(data['price']/data['total'], 2)
data['서비스'] = round(data['service']/data['total'], 2)
data['분위기'] = round(data['mood']/data['total'], 2)
data = data.drop('Unnamed: 0', axis=1)

In [ ]:
data.to_csv('/content/drive/Shareddrives/2022-1 KUBIG 추천시스템 장기프로젝트/total_naver3.csv', index=False, encoding='utf-8-sig')